In [2]:
pip install requests pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import pandas as pd

# USGS API URL'si
api_url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

# Parametreler
params = {
    "format": "geojson",
    "starttime": "2024-01-01",
    "endtime": "2024-01-31",
    "minmagnitude": 4.5
}

# API'den veri çekme
response = requests.get(api_url, params=params)

if response.status_code == 200:
    data = response.json()
    earthquakes = []
    
    # JSON verilerini işleme
    for feature in data['features']:
        properties = feature['properties']
        earthquakes.append({
            "Time": properties['time'],
            "Magnitude": properties['mag'],
            "Place": properties['place'],
            "Longitude": feature['geometry']['coordinates'][0],
            "Latitude": feature['geometry']['coordinates'][1],
            "Depth": feature['geometry']['coordinates'][2]
        })

    # Veriyi DataFrame'e çevirme
    df = pd.DataFrame(earthquakes)
    df['Time'] = pd.to_datetime(df['Time'], unit='ms')  # Zamanı düzenleme

    # CSV olarak kaydetme
    df.to_csv("earthquake_data.csv", index=False)
    print("Veriler başarılı bir şekilde kaydedildi.")
else:
    print("API çağrısında hata oluştu:", response.status_code)


/Users/gokcesoylu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Veriler başarılı bir şekilde kaydedildi.


In [4]:
import requests
import pandas as pd

# AFAD API URL'si
afad_url = "https://deprem.afad.gov.tr/event/get_event_list"

# API'den veri çekme
response = requests.get(afad_url)

if response.status_code == 200:
    data = response.json()
    earthquakes = []
    
    # JSON verilerini işleme
    for event in data:
        earthquakes.append({
            "Date": event['date'],
            "Time": event['time'],
            "Magnitude": event['magnitude'],
            "Latitude": event['latitude'],
            "Longitude": event['longitude'],
            "Depth": event['depth'],
            "Location": event['location']
        })

    # Veriyi DataFrame'e çevirme
    df = pd.DataFrame(earthquakes)

    # CSV olarak kaydetme
    df.to_csv("afad_earthquake_data.csv", index=False, encoding='utf-8')
    print("AFAD verileri başarılı bir şekilde kaydedildi.")
else:
    print("AFAD API çağrısında hata oluştu:", response.status_code)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
import requests

# AFAD API URL'si
afad_url = "https://deprem.afad.gov.tr/event/get_event_list"

# API'den veri çekme
response = requests.get(afad_url)

if response.status_code == 200:
    try:
        data = response.json()  # JSON verisini ayrıştır
        print("Veri başarıyla alındı.")
        print(data)  # Alınan JSON verisini yazdır
    except ValueError:
        print("API yanıtı JSON formatında değil.")
        print(response.text)  # Ham yanıtı yazdır
else:
    print("API çağrısında hata oluştu:", response.status_code)


API yanıtı JSON formatında değil.
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="utf-8" />
  <title>AFAD | deprem.gov.tr</title>
  <base href="/" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <meta name="description" content="Son Depremler - GÃ¼ncel deprem verilerini ve TÃ¼rkiye genelindeki depremlerle ilgili anlÄ±k bilgileri saÄlayan platform. Deprem bÃ¼yÃ¼klÃ¼kleri, konumlarÄ± ve tarihleri hakkÄ±nda detaylÄ± ve gÃ¼venilir bilgiler. AFAD Deprem Dairesi Son Deprem, Son Depremler, SonDeprem, SonDepremler, Last earthquakes, Latest earthquakes, Recent Earthquake, Earthquake, Deprem, Zelzele, KaÃ§ bÃ¼yÃ¼klÃ¼ÄÃ¼nde, Event, AvrupanÄ±n En bÃ¼yÃ¼k sismik aÄÄ±, En bÃ¼yÃ¼k sismik aÄ, ivmeÃ¶lÃ§er, kuvvetli yer hareketi, zayÄ±f yer hareketi, sismik sensÃ¶r, Deprem mi oldu, Son Depremler Listesi, Åimdi deprem mi oldu, Nerede deprem oldu">
  <link rel="icon" type="image/x-icon" href="./assets/img/afad_fav.png" />
  <link rel="stylesheet" href="https://

In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# AFAD Veri URL'si
afad_url = "https://deprem.afad.gov.tr/event/get_event_list"

# API'den veri çekme
response = requests.get(afad_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # HTML içeriğini analiz edin. Örnek: bir tabloyu seçmek
    table = soup.find('table')  # Tablo elemanını bul
    rows = table.find_all('tr')  # Tablodaki tüm satırları seç

    # Veriyi işleme
    data = []
    for row in rows[1:]:  # İlk satırı (başlık) atla
        cols = row.find_all('td')  # Sütunları bul
        cols = [col.text.strip() for col in cols]  # Metinleri çıkar
        data.append(cols)

    # DataFrame oluşturma
    columns = [col.text.strip() for col in rows[0].find_all('th')]  # Başlıklar
    df = pd.DataFrame(data, columns=columns)

    # CSV'ye kaydetme
    df.to_csv("afad_earthquake_data.csv", index=False, encoding='utf-8')
    print("AFAD verileri başarılı bir şekilde kaydedildi.")
else:
    print("API çağrısında hata oluştu:", response.status_code)


AttributeError: 'NoneType' object has no attribute 'find_all'

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

# AFAD Veri URL'si
afad_url = "https://deprem.afad.gov.tr/event/get_event_list"

# API'den veri çekme
response = requests.get(afad_url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')  # Tabloyu bulmaya çalış
    
    if table is not None:
        rows = table.find_all('tr')  # Tablodaki tüm satırları bul
        data = []

        for row in rows[1:]:  # İlk satırı (başlık) atla
            cols = row.find_all('td')  # Sütunları bul
            cols = [col.text.strip() for col in cols]  # Metinleri çıkar
            data.append(cols)

        # Tablo başlıklarını al
        columns = [col.text.strip() for col in rows[0].find_all('th')]
        
        # DataFrame oluşturma
        df = pd.DataFrame(data, columns=columns)

        # CSV'ye kaydetme
        df.to_csv("afad_earthquake_data.csv", index=False, encoding='utf-8')
        print("AFAD verileri başarılı bir şekilde kaydedildi.")
    else:
        print("HTML'de herhangi bir tablo bulunamadı.")
else:
    print("API çağrısında hata oluştu:", response.status_code)


HTML'de herhangi bir tablo bulunamadı.


In [8]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# AFAD Veri URL'si
afad_url = "https://deprem.afad.gov.tr/event/get_event_list"

# API'den veri çekme
response = requests.get(afad_url)

if response.status_code == 200:
    # XML verisini ayrıştırma
    root = ET.fromstring(response.content)

    data = []
    for event in root.findall('.//event'):  # 'event' elemanlarını bul
        data.append({
            "Date": event.find('date').text,
            "Time": event.find('time').text,
            "Magnitude": event.find('magnitude').text,
            "Latitude": event.find('latitude').text,
            "Longitude": event.find('longitude').text,
            "Depth": event.find('depth').text,
            "Location": event.find('location').text
        })

    # DataFrame oluşturma
    df = pd.DataFrame(data)

    # CSV'ye kaydetme
    df.to_csv("afad_earthquake_data.csv", index=False, encoding='utf-8')
    print("AFAD verileri başarılı bir şekilde kaydedildi.")
else:
    print("API çağrısında hata oluştu:", response.status_code)


ParseError: not well-formed (invalid token): line 14, column 14 (<string>)

In [9]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
response = requests.get(afad_url, headers=headers)

if response.status_code == 200:
    print(response.text)  # Verileri kontrol edin
else:
    print("Erişim hatası:", response.status_code)


<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="utf-8" />
  <title>AFAD | deprem.gov.tr</title>
  <base href="/" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <meta name="description" content="Son Depremler - GÃ¼ncel deprem verilerini ve TÃ¼rkiye genelindeki depremlerle ilgili anlÄ±k bilgileri saÄlayan platform. Deprem bÃ¼yÃ¼klÃ¼kleri, konumlarÄ± ve tarihleri hakkÄ±nda detaylÄ± ve gÃ¼venilir bilgiler. AFAD Deprem Dairesi Son Deprem, Son Depremler, SonDeprem, SonDepremler, Last earthquakes, Latest earthquakes, Recent Earthquake, Earthquake, Deprem, Zelzele, KaÃ§ bÃ¼yÃ¼klÃ¼ÄÃ¼nde, Event, AvrupanÄ±n En bÃ¼yÃ¼k sismik aÄÄ±, En bÃ¼yÃ¼k sismik aÄ, ivmeÃ¶lÃ§er, kuvvetli yer hareketi, zayÄ±f yer hareketi, sismik sensÃ¶r, Deprem mi oldu, Son Depremler Listesi, Åimdi deprem mi oldu, Nerede deprem oldu">
  <link rel="icon" type="image/x-icon" href="./assets/img/afad_fav.png" />
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/fon